# Coin toss game
Two gamblers are playing a coin toss game. Gambler A has (n+1) fair coins; B has n fair coins. What is the probability that A will have more heads than B if both flip all their coins?

Remove a coin of A and compare the number of heads in A's first n coins with B's n coins. Let N(A) be the number of heads in A's first n coins and N(B) be B's. There are three possible outcomes:

- $E_1$: $N(A) < N(B)$
- $E_2$: $N(A) = N(B)$
- $E_3$: $N(A) > N(B)$

The coins are fair, and by symmetry, $P(E_1) = P(E_3)$. Therefore, when adding one more coin, the probability that A will have more heads than B is $P(E_3) + P(E_2) \frac{1}{2} = P(E_3) + P(E_2) \cdot \frac{1}{2} = \frac{1}{2}$.

In [117]:
import random
import numba

n=100
N=10000
Awin=0

@numba.jit
def a_round():
    N_A=0
    N_B=0
    for i in range(n+1):
        if random.random() < 0.5:
            N_A += 1

    for i in range(n):
        if random.random() < 0.5:
            N_B += 1 

    return N_A, N_B

for i in range(N):
    N_A, N_B = a_round()
    if N_A > N_B:
        Awin += 1

print('the probility of A has more heads is: '+str(Awin/N))
print('the theoretical probility is: '+str(1/2))
print('the error is: '+str(100*abs(Awin/N-1/2))+'%')

the probility of A has more heads is: 0.5007
the theoretical probility is: 0.5
the error is: 0.07000000000000339%


# Drunk passenger
A line of 100 airline passengers are waiting to board a plane. They each hold a ticket to one of the 100 seats on that flight. For convenience, let's say that the n-th passenger in line has a ticket for seat number n. Being drunk, the first person in line picks a random seat (equally likely for each seat). All of the other passengers are sober and will go to their proper seats unless it is already occupied; in that case, they will randomly choose a free seat. You're person number 100. What is the probability that you end up in your seat (i.e., seat 100)?

Let's generalize this to n passengers. It can be observed that at each seating, there will be at most one displacement. In front of this displacement, everyone will sit in their assigned seats. Let $P_i$ be the probability that the n-th passenger will end up in their correct seat when the i-th passenger faces a random choice.

$$
\begin{aligned}
p_1&=\frac{1}{n}(1+P_2+P_3+\ldots+P_{n-1}) \\
P_i&=\frac{1}{n-i+1}(1+P_{i+1}+P_{i+2}+\ldots+P_{n-1}) \\
\implies&
\begin{cases}
(n-i+1)P_i&=1+P_{i+1}+P_{i+2}+\ldots+P_{n-1}&(1)\\
(n-i)P_{i+1}&=1+P_{i+2}+P_{i+3}+\ldots+P_{n-1}&(2)\\
\end{cases}
\\
(1)-(2)&\implies (n-i+1)P_i=(n-i+1)P_{i+1} \\
&\implies P_i \equiv P_{n-1}=\frac{1}{2}\\
\end{aligned}
$$
So the probability of the n-th person ending up in their seat is $\frac{1}{2}$

In [118]:
import random
import numba

n = 1000
N = 10000
nCorrect = 0

@numba.jit
def a_round():
    current = 0
    while current < n - 1:
        # There are n-current choices at the current position
        next = current + random.randint(1, n - current)
        current = (next + current) % n
        if current == 0:
            # If the first position is chosen before the last position, the last position is correct
            return True
    return False

for i in range(N):
    if a_round():
        nCorrect += 1

print('the probability of the last one being correct is: ' + str(nCorrect / N))
print('the theoretical probability is: ' + str(1 / 2))
print('the error is: ' + str(100*abs(nCorrect / N - 1 / 2)) + '%')

the probability of the last one being correct is: 0.5027
the theoretical probability is: 0.5
the error is: 0.27000000000000357%


In [119]:
import random

n = 1000
N = 10000
nCorrect = 0

def a_round():
    seats = [0] * n
    seats[random.randint(0, n-1)] = 1  # The first passenger randomly chooses a seat
    for i in range(1, n-1):
        if seats[i] == 0:
            seats[i] = 1  # Passenger sits in their assigned seat
        else:
            available_seats = [j for j in range(n) if seats[j] == 0]
            seats[random.choice(available_seats)] = 1  # Passenger randomly chooses an empty seat
    return seats[n-1] == 0  # Check if the last seat is empty

for i in range(N):
    if a_round():
        nCorrect += 1

print('the probability of the last one being correct is: ' + str(nCorrect / N))
print('the theoretical probability is: ' + str(1/2))
print('the error is: ' + str(100*abs(nCorrect / N - 1/2)) + '%')


the probability of the last one being correct is: 0.5019
the theoretical probability is: 0.5
the error is: 0.19000000000000128%


# N points on a circle
Given N points drawn randomly on the circumference of a circle, what is the probability that they are within a semicircle?

Consider a point as the leftmost point in a clockwise arrangement (N possibilities). The other points must be within the right semicircle of this point in a clockwise arrangement. Since each point's distribution is independent and random, the probability that all points are within the specified region is $\frac{1}{2^{N-1}}$. Thus, the total probability is $\frac{N}{2^{N-1}}$. Extending this, the probability of being within $1/M$ of the circumference is $\frac{N}{M^{N-1}}$.

In [120]:
import numpy as np
import numba

n = 100000  # Number of repetitions
N = 10  # Number of points
devideNum = 10000 * N  # Number of divisions
nCorrect = 0
M = 2

@numba.jit
def a_round():
    places = np.random.randint(0, devideNum, N)
    places = np.sort(places)
    max_gap = np.max(np.diff(places))
    if max_gap > devideNum / M or places[-1] - places[0] < devideNum / M:
        return True
    else:
        return False
    
@numba.jit
def getTheoryAns(N, M):
    return float(N / (pow(M, N - 1)))

for i in range(n):
    if a_round():
        nCorrect += 1
    
print('the probability is: ' + str(nCorrect / n))
print('the theoretical probability is: ' + str(getTheoryAns(N, M)))
print('the error is: ' + str(100 * (abs(nCorrect * pow(M, N - 1) - N * n)) / (N * n)) + '%')


the probability is: 0.01949
the theoretical probability is: 0.01953125
the error is: 0.2112%


# Chess tournament
A chess tournament has $2^n$ players with skills $1\gt2\gt\ldots\gt 2^n$. It is organized as a knockout tournament, so that after each round only the winner proceeds to the next round. Except for the final, opponents in each round are drawn at random. Let's also assume that when two players meet in a game, the player with better skills always wins. What's the probability that players 1 and 2 will meet in the final?

To solve this, we need to divide all players into two groups, each with $2^{n-1}$ players. The key is to ensure that players 1 and 2 are not in the same group. Each group is further divided into $2^{n-2}$ subgroups. For player 1, we can choose any group, then any subgroup, and finally an opponent. For player 2, we choose the other group, then any subgroup within that group, and finally an opponent. Therefore, the probability is:

$$
\begin{aligned}
P&=\frac{(C_2^1\cdot C_{2^{n-2}}^1\cdot C_{2^n-2}^1)\cdot(C_{2^{n-2}}^1\cdot C_{2^n-3}^1)\cdot(C_{2^n-4}^2\cdot C_{2^n-6}^2\cdot C_{2^n-8}^2\ldots C_2^2)}{C_{2^n}^2\cdot C_{2^n-2}^2\cdot C_{2^n-4}^2\ldots C_2^2} \\\\
&=\frac{2^n}{2(2^n-1)}
\end{aligned}
$$

In [121]:
import numpy as np

n=10 # 2^n players
num=pow(2,n)
N=10000 # number of rounds
count=0

def a_round():
    playerList=np.linspace(1,num,num)
    playerList=np.random.permutation(playerList)
    index1=np.where(playerList==1)[0][0]
    index2=np.where(playerList==2)[0][0]
    if index1<=num/2 and index2>num/2:
        return True
    elif index1>num/2 and index2<=num/2:
        return True
    else:
        return False
    
def getTheoryAns(n):
    return float(pow(2,n-1)/(pow(2,n)-1))
    
for i in range(N):
    if a_round():
        count+=1

print('the probability is: '+str(count/N))
print('the theoretical probability is: '+str(getTheoryAns(n)))
print('the error is: '+str(abs(count/N-getTheoryAns(n))/getTheoryAns(n)*100)+'%')


the probability is: 0.5015
the theoretical probability is: 0.5004887585532747
the error is: 0.2020507812499893%


# Application letters
Let's  denote by $E_i,i=1,2,3,\ldots,n$ the event that the i-th letter has the correct envelope. Then $P(\bigcup^n_{i=1}E_i)$ is the probability that at least one letter has the correct envelope and $1-P(\bigcup^n_{i=1}E_i)$ is the probability that all letters have the wrong envelopes. $P(\bigcup^n_{i=1}E_i)$ can be caculated using the Inclusion-Exclusion Principle:

$$
\begin{aligned}
P(\bigcup^n_{i=1}E_i)&=\sum^n_{i=1}P(E_i)-\sum_{i_1\lt i_2}P(E_{i_1}E_{i_2})+\ldots+(-1)^6P(E_1E_2\ldots E_5) \\
&=1-\frac{1}{2!}+\frac{1}{3!}-\frac{1}{4!}+\ldots+\frac{(-1)^{n+1}}{n}
\end{aligned}
$$

In [22]:
import numpy as np
import math
import numba

n=100 # number of letters
N=100000 # number of experiments
count=0

@numba.jit
def a_round():
    letterList = np.arange(1, n + 1)
    envelopeList = np.arange(1, n + 1)
    letterList = np.random.permutation(letterList)
    for i in range(n):
        if letterList[i] == envelopeList[i]:
            return False
    return True

def getTheoryAns(n):
    res=float(0)
    for i in range(1,n+1):
        res+=(pow(-1,i+1)*math.factorial(n)/math.factorial(i))
    return 1-res/math.factorial(n)

for i in range(N):
    if a_round():
        count+=1

print('the probability is: '+str(count/N))
print('the theoretical probability is: '+str(getTheoryAns(n)))
print('the error is: '+str(abs(count/N-getTheoryAns(n))/getTheoryAns(n)*100)+'%')

the probability is: 0.36788
the theoretical probability is: 0.3678794411714421
the error is: 0.0001519053514089647%


# Candies in a jar
You are taking out candies one by one from a jar that has 10 red candies, and 30 green candies in it. What is the probability that there are at least 1 blue candy and 1 green candy left in the jar when you have taken out all the candies?

Let $T_r,T_b,T_g$ be the number that the last red,blue,and green candies are taken out respectively. To have at least 1 blue candy and 1 green candy left when all the red candies are taken out, we need to have $T_r\lt T_b$ and $T_r\lt T_g$. In other words, we want to derive $P(T_r\lt T_b \bigcap T_r\lt T_g)$. There are two mutually exclusive events that satisfy $T_r\lt T_b$ and $T_r\lt T_g$ : 
1. $T_r\lt T_b\lt T_g$
2. $T_r\lt T_g\lt T_b$

$T_r\lt T_b\lt T_g$ means that the last candy is green($T_g=60$). Since each of the 60 candies are equally likely to be the last candy and among them 30 are green ones, we have $P(T_g=60)=\frac{30}{60}$. Conditioned on T_g=60, we need $P(T_r\lt T_b|T_g=60)$. Among the 30 red and blue candies, each candy is again equally likely to be the last candy and there are 20 blue candies, so $P(T_r\lt T_b|T_g=60)=\frac{20}{30}$ and $P(T_r\lt T_b\lt T_g)=\frac{30}{60}\times\frac{20}{30}$. Similarily, we have $P(T_r\lt T_g\lt T_b)=\frac{20}{60}\times\frac{30}{40}$

Hence,

$P(T_r\lt T_b \bigcap T_r\lt T_g)=\frac{7}{12}$

In [34]:
import numpy as np
import numba

r=10
b=20
g=30
N=10000
count=0

@numba.jit
def a_round():
    balls=np.zeros(r+b+g)
    balls[:r]=1
    balls[r:r+b]=2
    balls[r+b:]=3
    balls=np.random.permutation(balls)
    index1=np.where(balls==1)[0][-1]
    index2=np.where(balls==2)[0][-1]
    index3=np.where(balls==3)[0][-1]
    if index1<index2 and index1<index3:
        return True
    else:
        return False
    
def getTheoryAns(r,b,g):
    return float((g/(r+b+g))*(b/(r+b))+(b/(r+b+g)*(g/(r+g))))

for i in range(N):
    if a_round():
        count+=1

print('the probability is: '+str(count/N))
print('the theoretical probability is: '+str(getTheoryAns(r,b,g)))
print('the error is: '+str(abs(count/N-getTheoryAns(r,b,g))/getTheoryAns(r,b,g)*100)+'%')
        

the probability is: 0.5888
the theoretical probability is: 0.5833333333333333
the error is: 0.9371428571428684%


# Coin toss game
Two players, A and B, alternatively toss a fair coin(A tosses the coin first, then B tosses the coin, then A, then B,...). The Sequence of the heads and tails is recorded. If there is a head followed by a tail(HT subsequence), the game ends and the person who tosses the tail wins. What is the probability that A wins the game?

Let P(A) be the probability that A wins; then the probability that B wins is $P(B)=1-P(A)$. Let's condition P(A) on A's first toss, witch has 1/2 probability of H(heads) and 1/2 probability of T(tails).

$$
P(A)=\frac{1}{2}P(A|H)+\frac{1}{2}P(A|T) \\
$$

If A's first toss is T, then B essentially becomes the first to toss(An H is required for the HT subsequence). So we have $P(A|T)=P(B)=1-P(A)$

If A's first toss ends in H, lets further condition on B's first toss. B has 1/2 probability of getting T, in taht case A loses. For the 1/2 pprobability that B gets H, B essentially becomes the first one to toss an H. In that case, A has $(1-P(A|H))$ probability of winning. So $P(A|H)=\frac{1}{2}\times 0+\frac{1}{2}(1-P(A|H))\implies P(A|H)=\frac{1}{3}$

Combining all the available information  we have 

$$
P(A)=\frac{1}{2}\times\frac{1}{3}+\frac{1}{2}(1-P(a))\implies P(A)=\frac{4}{9} \\
$$

In [8]:
import numpy as np
import random
import numba

N=10000
count=0

@numba.jit
def a_round():
    # return A wins or not
    i=0
    lastCoin=0
    currentCoin=0
    while lastCoin!=1 or currentCoin!=0:
        lastCoin=currentCoin
        currentCoin=random.randint(0,1)
        i+=1
        
    return (i-1)&1==0

def getTheoryAns():
    return 4/9

for i in range(N):
    if a_round():
        count+=1

print('the probability is: '+str(count/N))
print('the theoretical probability is: '+str(getTheoryAns()))
print('the error is: '+str(abs(count/N-getTheoryAns())/getTheoryAns()*100)+'%')

the probability is: 0.4436
the theoretical probability is: 0.4444444444444444
the error is: 0.18999999999999573%


# Gamber's ruin problem
A gambler starts with an inital fortune of i dollars. On each successive game, the gambler wins \$1 with probability p,0 < p < 1,or loses \$1 with probability q=1-p. He will stop if he either accumulates N dollars or loses all his money. What is the probability that he will end up with N dollars?

$$
\begin{aligned}
&\text{Let } P(x) \text{ be the probability that the gambler loses all money when holding } x \text{ dollars} \\
&\text{It follows that } P(x) = pP(x+1) + qP(x-1) \\
&\text{The characteristic equation is } (p\lambda - (1-p))(\lambda - 1) = 0 \\
&\text{When } p \neq \frac{1}{2} \\
&\qquad\implies P(n) = A \cdot 1^n + B \cdot \left(\frac{1-p}{p}\right)^n \\
&\qquad\text{Substituting } P(0) = 1 \text{ and } P(N) = 0 \\
&\qquad\implies A = 1 - B, \quad B = \frac{1}{1 - \left(\frac{1-p}{p}\right)^N} \\
&\qquad\implies P(n) = 1 - \frac{1 - \left(\frac{1-p}{p}\right)^n}{1 - \left(\frac{1-p}{p}\right)^N} \\
&\text{When } p = \frac{1}{2} \\
&\qquad\implies P(x) = Ax + B \\
&\qquad\implies A = -\frac{1}{N}, \quad B = 1 \\
&\qquad\implies P(x) = 1 - \frac{x}{N} \\
&\implies \text{The probability of eventually winning } N \text{ dollars is} \\
&\qquad\begin{aligned}
P &= 1 - P(1) \\
&= \begin{cases}
\frac{1}{N}, & \text{if } p = \frac{1}{2} \\
\frac{1 - \left(\frac{1-p}{p}\right)}{1 - \left(\frac{1-p}{p}\right)^N}, & \text{if } p \neq \frac{1}{2} \\
\end{cases}
\end{aligned}
\end{aligned}
$$

In [13]:
import random
import numba

N=10000
count=0

@numba.jit
def a_round(p, M):
    money=1
    while money>0 and money<M:
        if random.random()<p:
            money+=1
        else:
            money-=1
    return money==M

def getTheoryAns(p, M):
    return 1/M if p==0.5 else (1-(1-p)/p)/(1-pow((1-p)/p,M))

for i in range(N):
    if a_round(0.5, 10):
        count+=1

print('when p=0.5')
print('the probability is: '+str(count/N))
print('the theoretical probability is: '+str(getTheoryAns(0.5, 10)))
print('the error is: '+str(abs(count/N-getTheoryAns(0.5, 10))/getTheoryAns(0.5, 10)*100)+'%')

count=0
for i in range(N):
    if a_round(0.6, 10):
        count+=1

print('when p=0.6')
print('the probability is: '+str(count/N))
print('the theoretical probability is: '+str(getTheoryAns(0.6, 10)))
print('the error is: '+str(abs(count/N-getTheoryAns(0.6, 10))/getTheoryAns(0.6, 10)*100)+'%')

when p=0.5
the probability is: 0.0978
the theoretical probability is: 0.1
the error is: 2.2000000000000073%
when p=0.6
the probability is: 0.3435
the theoretical probability is: 0.3392158552348125
the error is: 1.2629553421734774%


# Basketball scores
A basketball player is taking 100 free throws. She scorres on point if the ball passes through the hoop and zero point if she misses. She has scored on her first throw and missed on her second. For each of the following throw the probability of her scoring is the fraction of the throws she has made so far. For example, if she has scored 23 points after the 40th throw, the probability that she will score in the 41th throw is 23/40. After 100 throws(including the first and the second), what is the probability that she scores exactly 50 baskets?

## Solution1

Let $P_{i,j}$ be the probability of scoring $i$ points after $j$ throws.

$$
P_{i,j} = P_{i-1,j-1} \cdot \frac{i-1}{j-1} + P_{i,j-1} \cdot \left(1 - \frac{i}{j-1}\right)
$$

Given:
$$
P_{1,3} = \frac{1}{2}, \quad P_{2,3} = \frac{1}{2}, \quad P_{1,2} = 1, \quad j-1 \geq i \geq 1
$$

By calculation, we find:
$$
P_{1,4} = \frac{1}{3}, \quad P_{2,4} = \frac{1}{3}, \quad P_{3,4} = \frac{1}{3}
$$

Thus, we can conjecture that:
$$
P_{i,j} = \frac{1}{i-1}
$$

Substituting into the recurrence relation confirms this conjecture.

So the probability of having a total of 50 successes is $\frac{1}{99}$.

## Solution2
First, we need to understand the Beta-Binomial distribution.

The Beta-Binomial distribution is a discrete distribution that deals with varying success probabilities. When you have a Binomial distribution (i.e., repeated success-failure experiments with an unknown or varying success probability) and assume that this success probability follows a Beta distribution, you get the Beta-Binomial distribution.

Formula:

The probability mass function (PMF) of the Beta-Binomial distribution is:

$$ 
P(X=k) = \binom{n}{k} \frac{B(\alpha + k, \beta + n - k)}{B(\alpha, \beta)}
$$

where

- $X$ is the number of subsequent successes
- $k$ is the target number of subsequent successes
- $n$ is the total number of subsequent experiments
- $\alpha$ and $\beta$ represent the initial number of successes and failures, respectively
- $B(x, y)$ is the Beta function, $B(x, y) = \frac{\Gamma(x) \Gamma(y)}{\Gamma(x + y)}$

Substitute the values:

$$
\alpha = 1, \beta = 1 \ n = 98, k = 49 \ \implies P(X = 49) = \frac{1}{99}
$$

That is, the probability of having a total of 50 successes is $\frac{1}{99}$.

In [18]:
import random
import numba

N=1000000
count=0

@numba.jit
def a_round():
    score=1
    for i in range(2, 100):
        if random.random() < score / i:
            score+=1
        else:
            continue
    return score==50

def getTheoryAns():
    return 1/(100-1)

for i in range(N):
    if a_round():
        count+=1

print('the probability is: '+str(count/N))
print('the theoretical probability is: '+str(getTheoryAns()))
print('the error is: '+str(abs(count/N-getTheoryAns())/getTheoryAns()*100)+'%')  

the probability is: 0.010229
the theoretical probability is: 0.010101010101010102
the error is: 1.2670999999999937%
